### 一.构图
只需要准备俩RDD即可，一个表示存储顶点的ID及其属性信息，另一个存储源顶点srcId，目标顶点dstId，以及边上的属性信息即可;  

这里需要注意一下的是:  

1. ID列都需要是Long类型   
2. 构造边时需要用Edge类，而节点只需元组即可  
3. graph默认都是有向边

In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.graphx._
val sparkConf: SparkConf = new SparkConf().setMaster("local[2]")
val sc: SparkContext = new SparkContext(sparkConf)

sparkConf = org.apache.spark.SparkConf@4b8fc9ac
sc = org.apache.spark.SparkContext@257fbf29


org.apache.spark.SparkContext@257fbf29

In [2]:
val verticesRDD=sc.parallelize(Array((1L,"a"),(2L,"b"),(3L,"c"),(4L,"d"),(5L,"e")))
val edgesRDD=sc.parallelize(Array(Edge(1L,2L,"r1"),Edge(2L,3L,"r1"),Edge(3L,4L,"r2"),Edge(4L,5L,"r3"),Edge(3L,5L,"r2")))
val graph=Graph(verticesRDD,edgesRDD)

verticesRDD = ParallelCollectionRDD[0] at parallelize at <console>:31
edgesRDD = ParallelCollectionRDD[1] at parallelize at <console>:32
graph = org.apache.spark.graphx.impl.GraphImpl@41dd9ae8


org.apache.spark.graphx.impl.GraphImpl@41dd9ae8

### 二.查看出入度
出入度属性可以通过api直接查看，不过需要注意的是返回类型是RDD

In [3]:
//入度
graph.inDegrees.take(10)

Array((4,1), (2,1), (3,1), (5,2))

In [4]:
//出度
graph.outDegrees.take(10)

Array((4,1), (2,1), (1,1), (3,2))

### 三.添加/修改节点属性
上面我们为5个节点分别赋予了a~e的字符属性，我们还想为其添加新属性该怎么操作，两类方式：   

1.一类是直接对RDD进行操作即可，由于返回结果是RDD，所以需要再次调用Graph封装  
2.另一类是调用graph上的api，返回结果直接就是Graph

In [5]:
//rdd
Graph(verticesRDD.map(x=>(x._1,(x._2,1))),graph.edges).vertices.take(10)

Array((4,(d,1)), (2,(b,1)), (1,(a,1)), (3,(c,1)), (5,(e,1)))

In [6]:
//api
graph.mapVertices((vertexId,vertexAttr)=>(vertexAttr,1)).vertices.take(10)

Array((4,(d,1)), (2,(b,1)), (1,(a,1)), (3,(c,1)), (5,(e,1)))

### 四.添加/修改边上的属性
与节点操作类似，两类方式，RDD和调用api

In [7]:
//rdd
Graph(graph.vertices,edgesRDD.map(edge=>Edge(edge.srcId,edge.dstId,(edge.attr,1)))).edges.take(10)

Array(Edge(1,2,(r1,1)), Edge(2,3,(r1,1)), Edge(3,4,(r2,1)), Edge(3,5,(r2,1)), Edge(4,5,(r3,1)))

In [8]:
//api
graph.mapEdges(edge=>(edge.attr,1)).edges.take(10)

Array(Edge(1,2,(r1,1)), Edge(2,3,(r1,1)), Edge(3,4,(r2,1)), Edge(3,5,(r2,1)), Edge(4,5,(r3,1)))

### 五.同时利用节点和边的信息

如果想同时利用边的信息以及其关联的俩节点信息，如何处理呢，也是两种方式：   

1.RDD：利用边RDD的srcID去join 节点RDD，然后再使用dstID去join 节点RDD，这样得到的新RDD的每一行就都包括了所有的信息，然后再操作   
2.API：使用mapTriplets函数，它的输入包括5个属性，分别是边属性attr,源节点srcId,srcAttr,目标节点dstId,dstAttr,它的输出会直接更新为边attr  

通过这个api，我们可以做一些复杂点的操作，比如我们想给边添加一个新属性，生成该属性的规则是：如果源节点或者目标节点的属性为"e",或者边的属性为"r2"则返回1,否则返回0

In [9]:
graph.mapTriplets(edge=>{
    val newValue=if(edge.srcAttr=="e" || edge.dstAttr=="e" || edge.attr=="r2") 1 else 0
    (edge.attr,newValue)}
).edges.take(10)

Array(Edge(1,2,(r1,0)), Edge(2,3,(r1,0)), Edge(3,4,(r2,1)), Edge(3,5,(r2,1)), Edge(4,5,(r3,1)))